## Chapter9.Control Abstraction




### Reducing code duplication




In [2]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles

    def filesEnding(query: String) =
      for (file <- filesHere; if file.getName.endsWith(query))
        yield file
  }

hi


In [3]:
def filesContaining(query: String) =
    for (file <- filesHere; if file.getName.contains(query))
      yield file

In [4]:
def filesRegex(query: String) =
    for (file <- filesHere; if file.getName.matches(query))
      yield file

In [5]:
 def filesMatching(query: String, method) =
    for (file <- filesHere; if file.getName.method(query))
      yield file


In [6]:
def filesMatching(query: String,
    matcher: (String, String) => Boolean) = {

    for (file <- filesHere; if matcher(file.getName, query))
      yield file
  }

In [7]:
def filesEnding(query: String) =
    filesMatching(query, _.endsWith(_))

def filesContaining(query: String) =
    filesMatching(query, _.contains(_))

def filesRegex(query: String) =
    filesMatching(query, _.matches(_))


In [8]:
object FileMatcher {
    private def filesHere = (new java.io.File(".")).listFiles

    private def filesMatching(matcher: String => Boolean) =
      for (file <- filesHere; if matcher(file.getName))
        yield file
  
    def filesEnding(query: String) =
      filesMatching(_.endsWith(query))
  
    def filesContaining(query: String) =
      filesMatching(_.contains(query))
  
    def filesRegex(query: String) =
      filesMatching(_.matches(query))
  }


### Simplifying client code




In [10]:
 def containsNeg(nums: List[Int]): Boolean = {
    var exists = false
    for (num <- nums)
      if (num < 0)
        exists = true
    exists
}
containsNeg(List(1, 2, 3, 4))
containsNeg(List(1, 2, -3, 4))
containsNeg(Nil)
containsNeg(List(0, -1, -2))

true

In [11]:
def containsOdd(nums: List[Int]): Boolean = {
    var exists = false
    for (num <- nums)
      if (num % 2 == 1)
        exists = true
    exists
}


containsNeg(List(1, 2, -3, 4))

true

In [12]:
def containsOdd(nums: List[Int]) = nums.exists(_ % 2 == 1)
containsNeg(List(1, 2, -3, 4))

true

### Currying




In [14]:
def plainOldSum(x: Int, y: Int) = x + y
plainOldSum(1, 2)

3

In [16]:
def curriedSum(x: Int)(y: Int) = x + y
curriedSum(1)(2)

3

In [17]:
def first(x: Int) = (y: Int) => x + y
val second = first(1)
second(2)

3

In [18]:
val onePlus = curriedSum(1)_
onePlus(2)

3

### Short forms of function literals




In [20]:
val twoPlus = curriedSum(2)_
twoPlus(2)

4

### Writing new control structures




In [22]:
def twice(op: Double => Double, x: Double) = op(op(x))
 twice(_ + 1, 5)

7.0

In [23]:
import java.io.File
def withPrintWriter(file: File, op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
      op(writer)
    } finally {
      writer.close()
    }
}
withPrintWriter(new File("date.txt"),
    writer => writer.println(new java.util.Date)
)

Error: not found: type PrintWriter (104)Error: not found: type PrintWriter (57)

In [24]:
def withPrintWriter(file: File)(op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
      op(writer)
    } finally {
      writer.close()
    }
}

In [25]:
 def withPrintWriter(file: File)(op: PrintWriter => Unit) = {
    val writer = new PrintWriter(file)
    try {
      op(writer)
    } finally {
      writer.close()
    }
}

In [26]:
val file = new File("date.txt")

  withPrintWriter(file) { writer =>
    writer.println(new java.util.Date)
  }


### By-name parameters




In [28]:
withPrintWriter(file) { writer =>
    writer.println(new java.util.Date)
}


Error: not found: value withPrintWriter (0)Error: not found: value file (16)

In [29]:

var assertionsEnabled = true

def myAssert(predicate: () => Boolean) =
    if (assertionsEnabled && !predicate())
      throw new AssertionError
myAssert(() => 5 > 3)
def boolAssert(predicate: Boolean) =
    if (assertionsEnabled && !predicate)
      throw new AssertionError
boolAssert(5 > 3)
def byNameAssert(predicate: => Boolean) =
    if (assertionsEnabled && !predicate)
      throw new AssertionError
 val x = 5
 assertionsEnabled = false
 boolAssert(x / 0 == 0)
 byNameAssert(x / 0 == 0)

java.lang.ArithmeticException: / by zero